In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

# READING FILES

## SWAP

In [2]:
query = pd.read_csv(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2022\01. JANUARY\SWAP 2022-01-31.csv'
                     ,dtype={'bank_account_number':object,'report_id':object})
query.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap
0,0000d758-84b4-4cb0-a165-4f0d3c44864d,MAHARISHI PANTJALI SA DE CV,maharishiclip@gmail.com,75HNWQ7W8,2022-01-31 00:00:00,"66,652.53",PENDING,"Banco Regional de Monterrey, S.A.",2022-01-31 09:12:58,MAHARISHI PANTJALI S.A. DE C.V.,'058375000003160831,2022-01-31 09:03:18,"MAHARISHI PANTJALI S.A. DE C.V.,,66652.53,0583..."
1,0000e953-6b2c-44d0-a798-9dac1abf24ea,Super Kandy,yanellysegovia18@gmail.com,4YYNJCD2T,2022-01-31 00:00:00,"5,039.84",PENDING,"Banco Nacional de México, S.A.",2022-01-31 09:12:23,Ana Yanelly Puente Gutierrez,'002580701334955664,2022-01-31 09:03:18,"Ana Yanelly Puente Gutierrez,,5039.84,00258070..."
2,00023bd5-6149-4ef9-95be-edb61d22588d,LAS PEKES,alebarradas1@hotmail.com,DBM9TZH9J,2022-01-31 00:00:00,43.12,PENDING,"BanCoppel, S.A.",2022-01-31 09:11:47,Virginia Alejandra Barradas Prez,'137463103868892825,2022-01-31 09:03:18,"Virginia Alejandra Barradas Prez,,43.12,137463..."
3,000255c1-03db-49c8-993d-2ce223408b0a,La Mision,micaelaguerrero3531@gmail.com,DKLFCG7B7,2022-01-31 00:00:00,61.81,PENDING,"Banco Regional de Monterrey, S.A.",2022-01-31 09:12:58,Micaela Guerrero,'058680000007494201,2022-01-31 09:03:18,"Micaela Guerrero,,61.81,058680000007494201,CLI..."
4,0003c12e-9130-4af9-b4be-dba67bfeab51,Nakama Sushi,nakama.sushi.pfp@gmail.com,XQ6WWFZNM,2022-01-31 00:00:00,182.07,PENDING,"BBVA Bancomer, S.A.",2022-01-31 09:11:47,Chtistian David Guzman Renteria,'012180015984386028,2022-01-31 09:03:18,"Chtistian David Guzman Renteria,,182.07,012180..."


In [3]:
ae = pd.read_csv(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2022\01. JANUARY\SwapDOWNLOAD 2022-01-31.csv'
                 ,names=['Nombre_download','Blank','amount_download','CLABE_download','report_id']
                 ,dtype={'CLABE_download':object,'report_id':object},skiprows=[0])
ae['report_id'] = ae['report_id'].str[5:].replace(' ', '',regex=True)
ae['report_id'] = ae['report_id'].replace(' ', '',regex=True)
ae.head()

,Nombre_download,Blank,amount_download,CLABE_download,report_id
0,Gabriela Cruz Aragon,nan,8.00,012180011642955876,VBRNQFSJV
1,Juan Jose Abarca Barreto,nan,"1,830.23",012040011224470291,2Z4R2SP2X
2,Victor Hugo Flores Munoz,nan,"1,437.36",002180902707725277,QYWYS4NWR
3,JPA SOLUCIONES AEREAS S A DE C V,nan,"30,320.63",012691001161237406,5WR3TNVFT
4,JESSON FERNANDO MARTINEZ ROBLES,nan,59.41,072060010927954789,822PPZ8LT


In [4]:
query.shape

(58031, 13)

In [5]:
ae.shape

(58031, 5)

#  PAGOS DUPLICADOS

In [6]:
duplicates_query = query[query.duplicated(['report_id'],keep=False)]
duplicates_query.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap


In [7]:
duplicates_file_sent = ae[ae.duplicated(['report_id'],keep=False)]
duplicates_file_sent.head()

,Nombre_download,Blank,amount_download,CLABE_download,report_id


# VALIDACION REGISTROS

In [8]:
valid_1 = query.merge(ae[['report_id','CLABE_download']] ,on='report_id', how='left')
valid_1['CLABE_download'].fillna('NOT FOUND',inplace=True)
valid_1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap,CLABE_download
0,0000d758-84b4-4cb0-a165-4f0d3c44864d,MAHARISHI PANTJALI SA DE CV,maharishiclip@gmail.com,75HNWQ7W8,2022-01-31 00:00:00,"66,652.53",PENDING,"Banco Regional de Monterrey, S.A.",2022-01-31 09:12:58,MAHARISHI PANTJALI S.A. DE C.V.,'058375000003160831,2022-01-31 09:03:18,"MAHARISHI PANTJALI S.A. DE C.V.,,66652.53,0583...",058375000003160831
1,0000e953-6b2c-44d0-a798-9dac1abf24ea,Super Kandy,yanellysegovia18@gmail.com,4YYNJCD2T,2022-01-31 00:00:00,"5,039.84",PENDING,"Banco Nacional de México, S.A.",2022-01-31 09:12:23,Ana Yanelly Puente Gutierrez,'002580701334955664,2022-01-31 09:03:18,"Ana Yanelly Puente Gutierrez,,5039.84,00258070...",002580701334955664
2,00023bd5-6149-4ef9-95be-edb61d22588d,LAS PEKES,alebarradas1@hotmail.com,DBM9TZH9J,2022-01-31 00:00:00,43.12,PENDING,"BanCoppel, S.A.",2022-01-31 09:11:47,Virginia Alejandra Barradas Prez,'137463103868892825,2022-01-31 09:03:18,"Virginia Alejandra Barradas Prez,,43.12,137463...",137463103868892825
3,000255c1-03db-49c8-993d-2ce223408b0a,La Mision,micaelaguerrero3531@gmail.com,DKLFCG7B7,2022-01-31 00:00:00,61.81,PENDING,"Banco Regional de Monterrey, S.A.",2022-01-31 09:12:58,Micaela Guerrero,'058680000007494201,2022-01-31 09:03:18,"Micaela Guerrero,,61.81,058680000007494201,CLI...",058680000007494201
4,0003c12e-9130-4af9-b4be-dba67bfeab51,Nakama Sushi,nakama.sushi.pfp@gmail.com,XQ6WWFZNM,2022-01-31 00:00:00,182.07,PENDING,"BBVA Bancomer, S.A.",2022-01-31 09:11:47,Chtistian David Guzman Renteria,'012180015984386028,2022-01-31 09:03:18,"Chtistian David Guzman Renteria,,182.07,012180...",012180015984386028


In [9]:
valid_1.shape

(58031, 14)

In [10]:
check_validation = valid_1[(valid_1['CLABE_download'] != 'NOT FOUND')]
check_validation.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap,CLABE_download
0,0000d758-84b4-4cb0-a165-4f0d3c44864d,MAHARISHI PANTJALI SA DE CV,maharishiclip@gmail.com,75HNWQ7W8,2022-01-31 00:00:00,"66,652.53",PENDING,"Banco Regional de Monterrey, S.A.",2022-01-31 09:12:58,MAHARISHI PANTJALI S.A. DE C.V.,'058375000003160831,2022-01-31 09:03:18,"MAHARISHI PANTJALI S.A. DE C.V.,,66652.53,0583...",058375000003160831
1,0000e953-6b2c-44d0-a798-9dac1abf24ea,Super Kandy,yanellysegovia18@gmail.com,4YYNJCD2T,2022-01-31 00:00:00,"5,039.84",PENDING,"Banco Nacional de México, S.A.",2022-01-31 09:12:23,Ana Yanelly Puente Gutierrez,'002580701334955664,2022-01-31 09:03:18,"Ana Yanelly Puente Gutierrez,,5039.84,00258070...",002580701334955664
2,00023bd5-6149-4ef9-95be-edb61d22588d,LAS PEKES,alebarradas1@hotmail.com,DBM9TZH9J,2022-01-31 00:00:00,43.12,PENDING,"BanCoppel, S.A.",2022-01-31 09:11:47,Virginia Alejandra Barradas Prez,'137463103868892825,2022-01-31 09:03:18,"Virginia Alejandra Barradas Prez,,43.12,137463...",137463103868892825
3,000255c1-03db-49c8-993d-2ce223408b0a,La Mision,micaelaguerrero3531@gmail.com,DKLFCG7B7,2022-01-31 00:00:00,61.81,PENDING,"Banco Regional de Monterrey, S.A.",2022-01-31 09:12:58,Micaela Guerrero,'058680000007494201,2022-01-31 09:03:18,"Micaela Guerrero,,61.81,058680000007494201,CLI...",058680000007494201
4,0003c12e-9130-4af9-b4be-dba67bfeab51,Nakama Sushi,nakama.sushi.pfp@gmail.com,XQ6WWFZNM,2022-01-31 00:00:00,182.07,PENDING,"BBVA Bancomer, S.A.",2022-01-31 09:11:47,Chtistian David Guzman Renteria,'012180015984386028,2022-01-31 09:03:18,"Chtistian David Guzman Renteria,,182.07,012180...",012180015984386028


In [11]:
check_validation.shape

(58031, 14)

In [12]:
check_validation_2 = valid_1[(valid_1['CLABE_download'] == 'NOT FOUND')]
check_validation_2.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap,CLABE_download


In [13]:
check_validation_2.shape

(0, 14)

In [14]:
check_validation.columns

Index(['merchant_id', 'merchant_name', 'user_email', 'report_id',
       'report_date_local', 'amount', 'status', 'bank_name',
       'payment_date_local', 'bank_account_owner_name', 'bank_account_number',
       'batch_created_date', 'swap', 'CLABE_download'],
      dtype='object')

In [15]:
check_validation_final = check_validation[['merchant_id','merchant_name','user_email','report_id',
       'report_date_local','amount','status','bank_name',
       'payment_date_local','bank_account_owner_name','bank_account_number',
       'swap']]
check_validation_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,swap
0,0000d758-84b4-4cb0-a165-4f0d3c44864d,MAHARISHI PANTJALI SA DE CV,maharishiclip@gmail.com,75HNWQ7W8,2022-01-31 00:00:00,"66,652.53",PENDING,"Banco Regional de Monterrey, S.A.",2022-01-31 09:12:58,MAHARISHI PANTJALI S.A. DE C.V.,'058375000003160831,"MAHARISHI PANTJALI S.A. DE C.V.,,66652.53,0583..."
1,0000e953-6b2c-44d0-a798-9dac1abf24ea,Super Kandy,yanellysegovia18@gmail.com,4YYNJCD2T,2022-01-31 00:00:00,"5,039.84",PENDING,"Banco Nacional de México, S.A.",2022-01-31 09:12:23,Ana Yanelly Puente Gutierrez,'002580701334955664,"Ana Yanelly Puente Gutierrez,,5039.84,00258070..."
2,00023bd5-6149-4ef9-95be-edb61d22588d,LAS PEKES,alebarradas1@hotmail.com,DBM9TZH9J,2022-01-31 00:00:00,43.12,PENDING,"BanCoppel, S.A.",2022-01-31 09:11:47,Virginia Alejandra Barradas Prez,'137463103868892825,"Virginia Alejandra Barradas Prez,,43.12,137463..."
3,000255c1-03db-49c8-993d-2ce223408b0a,La Mision,micaelaguerrero3531@gmail.com,DKLFCG7B7,2022-01-31 00:00:00,61.81,PENDING,"Banco Regional de Monterrey, S.A.",2022-01-31 09:12:58,Micaela Guerrero,'058680000007494201,"Micaela Guerrero,,61.81,058680000007494201,CLI..."
4,0003c12e-9130-4af9-b4be-dba67bfeab51,Nakama Sushi,nakama.sushi.pfp@gmail.com,XQ6WWFZNM,2022-01-31 00:00:00,182.07,PENDING,"BBVA Bancomer, S.A.",2022-01-31 09:11:47,Chtistian David Guzman Renteria,'012180015984386028,"Chtistian David Guzman Renteria,,182.07,012180..."


In [16]:
check_validation_final.shape

(58031, 12)

# EXCEL

In [17]:
# check_validation_final.to_csv(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\05. MAY\SWAP 2021-05-05.csv',index=False)

In [18]:
writer = pd.ExcelWriter(r'DIFF 2022-01-31.xlsx', engine='xlsxwriter', options={'encoding':'UTF-8'})
check_validation.to_excel(writer, sheet_name='DATA MATCH',index=False)
check_validation_2.to_excel(writer, sheet_name='DATA NOT MATCH',index=False)
writer.save()
writer.close()